# Settings

## Constant

In [1]:
import pytz
import os


DEFAULT_TZ = pytz.FixedOffset(540)  # GMT+09:00; Asia/Seoul

PATH_DATA = './data'
PATH_ESM = os.path.join(PATH_DATA, 'EsmResponse.csv')
PATH_PARTICIPANT = os.path.join(PATH_DATA, 'UserInfo.csv')
PATH_SENSOR = os.path.join(PATH_DATA, 'Sensor')

PATH_INTERMEDIATE = './intermediate'

DATA_TYPES = {
#     'EDA': 'EDA',
    'HR': 'HRT',
#     'RRI': 'RRI',
#     'SkinTemperature': 'SKT',
}


## Utility Functions

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import cloudpickle
import ray
from datetime import datetime
from contextlib import contextmanager
import warnings
import time


def load(path: str):
    with open(path, mode='rb') as f:
        return cloudpickle.load(f)
    
def dump(obj, path: str):
    with open(path, mode='wb') as f:
        cloudpickle.dump(obj, f)
    
def log(msg: any):
    print('[{}] {}'.format(datetime.now().strftime('%y-%m-%d %H:%M:%S'), msg))

def summary(x):
    x = np.asarray(x)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        n = len(x)
        # Here, uppercase np.dtype.kind corresponds to non-numeric data.
        # Also, we view the boolean data as dichotomous categorical data.
        if x.dtype.kind.isupper() or x.dtype.kind == 'b': 
            cnt = pd.Series(x).value_counts(dropna=False)
            card = len(cnt)
            cnt = cnt[:20]                
            cnt_str = ', '.join([f'{u}:{c}' for u, c in zip(cnt.index, cnt)])
            if card > 30:
                cnt_str = f'{cnt_str}, ...'
            return {
                'n': n,
                'cardinality': card,
                'value_count': cnt_str
            }
        else: 
            x_nan = x[np.isnan(x)]
            x_norm = x[~np.isnan(x)]
            
            tot = np.sum(x_norm)
            m = np.mean(x_norm)
            me = np.median(x_norm)
            s = np.std(x_norm, ddof=1)
            l, u = np.min(x_norm), np.max(x)
            conf_l, conf_u = st.t.interval(0.95, len(x_norm) - 1, loc=m, scale=st.sem(x_norm))
            n_nan = len(x_nan)
            
            return {
                'n': n,
                'sum': tot,
                'mean': m,
                'SD': s,
                'med': me,
                'range': (l, u),
                'conf.': (conf_l, conf_u),
                'nan_count': n_nan
            }


@contextmanager
def on_ray(*args, **kwargs):
    try:
        if ray.is_initialized():
            ray.shutdown()
        ray.init(*args, **kwargs)
        yield None
    finally:
        ray.shutdown()

2025-06-05 22:32:12,059	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Settings for R

In [3]:
# %load_ext rpy2.ipython

In [4]:
# %%R

# library(tidyverse)
# library(ggforce)
# library(ggpubr)
# library(showtext)
# library(rmcorr)
# library(patchwork)

# # font_add_google(
# #     name='Source Serif Pro',
# #     family='ssp',
# #     db_cache=FALSE
# # )

# showtext_auto()

# THEME_DEFAULT <- theme_bw(
#     base_size=10,
#     base_family='ssp',
# ) + theme(
#         axis.title.x=element_text(colour='grey20', size=10, face='bold'),
#         axis.title.y=element_text(colour='grey20', size=10, face='bold'),
#         axis.text.x=element_text(colour='grey20', size=10),
#         axis.text.y=element_text(colour='grey20', size=10),
#         strip.text.x=element_text(colour='grey20', size=10, face='bold'),
#         strip.text.y=element_text(colour='grey20', size=10, face='bold'),
#         legend.background=element_blank(),
#         legend.title=element_text(colour='grey20', size=10, face='bold'),
#         legend.text=element_text(colour='grey20', size=10),
#         legend.position='top',
#         legend.box.spacing= unit(0, 'cm'),
#         plot.subtitle=element_text(colour='grey20', size=10, hjust=.5),
#     )


# Dataset Overview

## Participants

In [5]:
import pandas as pd
import os


PARTICIPANTS = pd.read_csv(PATH_PARTICIPANT).set_index('pcode').assign(
    particpationStartDateTime=lambda x: pd.to_datetime(
        x['participationStartDate'], format='%Y-%m-%d'
    ).dt.tz_localize(DEFAULT_TZ)
)
PARTICIPANTS.head()

,participationStartDate,age,gender,openness,conscientiousness,neuroticism,extraversion,agreeableness,PSS,PHQ,GHQ,particpationStartDateTime
pcode,,,,,,,,,,,,
P01,2019-05-08,27,M,11,11,3,4,13,13,0,1,2019-05-08 00:00:00+09:00
P02,2019-05-08,21,M,14,5,12,14,5,27,6,18,2019-05-08 00:00:00+09:00
P03,2019-05-08,24,F,10,15,8,7,11,18,2,6,2019-05-08 00:00:00+09:00
P04,2019-05-08,23,M,12,11,8,6,11,20,1,9,2019-05-08 00:00:00+09:00
P05,2019-05-08,27,F,10,11,13,10,6,25,14,9,2019-05-08 00:00:00+09:00


Belows are some demographics:

In [6]:
for c in PARTICIPANTS.columns:
    print(f'- {c}:', summary(PARTICIPANTS[c]))

- participationStartDate: {'n': 77, 'cardinality': 3, 'value_count': '2019-05-08:27, 2019-05-16:25, 2019-04-30:25'}
- age: {'n': 77, 'sum': 1686, 'mean': 21.896103896103895, 'SD': 3.8613619617422406, 'med': 21.0, 'range': (17, 38), 'conf.': (21.01968223607122, 22.77252555613657), 'nan_count': 0}
- gender: {'n': 77, 'cardinality': 2, 'value_count': 'M:53, F:24'}
- openness: {'n': 77, 'sum': 787, 'mean': 10.220779220779221, 'SD': 2.8956563505732467, 'med': 11.0, 'range': (3, 15), 'conf.': (9.563545847995773, 10.87801259356267), 'nan_count': 0}
- conscientiousness: {'n': 77, 'sum': 820, 'mean': 10.64935064935065, 'SD': 2.3662441579221882, 'med': 11.0, 'range': (5, 15), 'conf.': (10.112279104782713, 11.186422193918586), 'nan_count': 0}
- neuroticism: {'n': 77, 'sum': 618, 'mean': 8.025974025974026, 'SD': 2.6900108881310953, 'med': 8.0, 'range': (3, 14), 'conf.': (7.4154164477308075, 8.636531604217245), 'nan_count': 0}
- extraversion: {'n': 77, 'sum': 703, 'mean': 9.12987012987013, 'SD': 3.

## Labels (via ESM)

In [7]:
import pandas as pd
import os


LABELS = pd.read_csv(PATH_ESM).set_index(
    ['pcode']
)
LABELS.head()

,responseTime,scheduledTime,valence,arousal,attention,stress,duration,disturbance,change
pcode,,,,,,,,,
P01,1557278103000,NaN,0,0,0,-1,20.0,3,-2
P01,1557278986000,1.557279e+12,-3,3,3,3,5.0,-1,-3
P01,1557281772000,1.557282e+12,-3,-2,2,2,15.0,3,-2
P01,1557287138000,NaN,2,-1,2,0,15.0,1,-1
P01,1557291117000,NaN,3,3,3,-3,20.0,1,0


In [8]:
for c in LABELS.columns:
    print(f'- {c}:', summary(LABELS[c]))

- responseTime: {'n': 5582, 'sum': 8694314195328000, 'mean': 1557562557385.8833, 'SD': 590915040.4254278, 'med': 1557562969500.0, 'range': (1556582982000, 1558545246000), 'conf.': (1557547052362.8618, 1557578062408.9048), 'nan_count': 0}
- scheduledTime: {'n': 5582, 'sum': 5175814282500000.0, 'mean': 1557572760306.9517, 'SD': 591697484.8543198, 'med': 1557565860000.0, 'range': (1556586120000.0, nan), 'conf.': (1557552635074.4736, 1557592885539.4297), 'nan_count': 2259}
- valence: {'n': 5582, 'sum': 3665, 'mean': 0.6565747044070226, 'SD': 1.4184297545899174, 'med': 1.0, 'range': (-3, 3), 'conf.': (0.6193565182132938, 0.6937928906007513), 'nan_count': 0}
- arousal: {'n': 5582, 'sum': -529, 'mean': -0.09476890003582945, 'SD': 1.6675313128774563, 'med': 0.0, 'range': (-3, 3), 'conf.': (-0.13852326339835566, -0.051014536673303246), 'nan_count': 0}
- attention: {'n': 5582, 'sum': 2236, 'mean': 0.4005732712289502, 'SD': 1.6113242733571864, 'med': 1.0, 'range': (-3, 3), 'conf.': (0.35829372468

Belows are some demographics:

In [9]:
inst = LABELS.groupby('pcode').count().iloc[:, -1]
inst_sch = LABELS.loc[lambda x: ~x['scheduledTime'].isna(), :].groupby('pcode').count().iloc[:, -1]
inst_vol = LABELS.loc[lambda x: x['scheduledTime'].isna(), :].groupby('pcode').count().iloc[:, -1]
resp_time = LABELS.assign(
    timestamp=lambda x: pd.to_datetime(x['responseTime'], unit='ms', utc=True).dt.tz_convert(DEFAULT_TZ)
)
sam = np.concatenate([
    (resp_time.loc[p, 'timestamp'].array - resp_time.loc[p, 'timestamp'].array.shift(1)).dropna().total_seconds()
    for p in LABELS.index.unique()
])

print('- # Inst.:', summary(inst))
print('- # Inst. - Scheduled:', summary(inst_sch))
print('- # Inst. - Voluntary:', summary(inst_vol))
print('- Samp. period:', summary(sam))
for c in LABELS.columns:
    print(f'- {c}:', summary(LABELS[c]))

- # Inst.: {'n': 77, 'sum': 5582, 'mean': 72.49350649350649, 'SD': 16.02270048911147, 'med': 74.0, 'range': (20, 110), 'conf.': (68.85679957506535, 76.13021341194762), 'nan_count': 0}
- # Inst. - Scheduled: {'n': 76, 'sum': 3323, 'mean': 43.723684210526315, 'SD': 19.36291898394835, 'med': 43.5, 'range': (3, 83), 'conf.': (39.29906768289902, 48.14830073815361), 'nan_count': 0}
- # Inst. - Voluntary: {'n': 77, 'sum': 2259, 'mean': 29.337662337662337, 'SD': 16.297893300742235, 'med': 27.0, 'range': (2, 74), 'conf.': (25.6384943127028, 33.03683036262187), 'nan_count': 0}
- Samp. period: {'n': 5505, 'sum': 42240670.0, 'mean': 7673.146230699364, 'SD': 13193.471538029606, 'med': 3090.0, 'range': (1.0, 136446.0), 'conf.': (7324.548923384188, 8021.743538014541), 'nan_count': 0}
- responseTime: {'n': 5582, 'sum': 8694314195328000, 'mean': 1557562557385.8833, 'SD': 590915040.4254278, 'med': 1557562969500.0, 'range': (1556582982000, 1558545246000), 'conf.': (1557547052362.8618, 1557578062408.9048)

### Plot

In [10]:
data = LABELS.loc[
    :, lambda x: ~x.columns.isin(['responseTime', 'scheduledTime'])
]

In [11]:
# %%R -i data -w 16 -h 6 -u cm

# data <- data %>% pivot_longer(
#     cols = c('valence', 'arousal', 'attention', 'stress', 'duration', 'disturbance', 'change'),
#     names_to = 'metric'
# )

# p_rest <- ggplot(
#     data %>% filter(metric != 'duration'), aes(x=metric, y=value)
# ) + geom_boxplot(
# ) + geom_point(
#     data = data %>% filter(
#         metric != 'duration'
#     ) %>% group_by(
#         metric
#     ) %>% summarise(
#         mean = mean(value, na.rm=TRUE)
#     ),
#     mapping=aes(x=metric, y=mean),
#     shape=21,
#     stroke=1,
#     size=2,
#     fill='white'
# ) + scale_x_discrete(
#     name=NULL,
#     limits=c('valence', 'arousal', 'stress', 'attention', 'disturbance', 'change'),
#     labels=c('Valence', 'Arousal', 'Stress', 'Attent.', 'Disturb.', 'Change'),
# ) + scale_y_continuous(
#     name='Response',
#     breaks=-3:3
# ) + THEME_DEFAULT

# p_duration <- ggplot(
#     data %>% filter(metric == 'duration'), aes(x=metric, y=value)
# ) + geom_boxplot(
# ) + geom_point(
#     data = data %>% filter(
#         metric == 'duration'
#     ) %>% group_by(
#         metric
#     ) %>% summarise(
#         mean = mean(value, na.rm=TRUE)
#     ),
#     mapping=aes(x=metric, y=mean),
#     shape=21,
#     stroke=1,
#     size=2,
#     fill='white'
# )+ scale_x_discrete(
#     name=NULL,
#     limits=c('duration'),
#     labels=c('Duration'),
# ) + scale_y_continuous(
#     name=NULL,
#     breaks=seq(from=5, to=60, by=10)
# ) + THEME_DEFAULT

# p <- p_rest + p_duration + plot_layout(widths=c(4, 0.8))
# ggsave('./fig/dist-labels.pdf', plot=p, width=16, height=6, unit='cm', device=cairo_pdf)
# print(p)

### Correlation

Because each participant reported their labels multiple times (i.e., repeated measure), repeated measure correlation between affect labels were used.

In [12]:
data = LABELS.reset_index()[[
    'pcode', 'valence', 'arousal', 'stress', 'attention', 'disturbance', 'change'
]]

In [13]:
# %%R -i data 

# com <- combn(c('valence', 'arousal', 'stress', 'attention', 'disturbance', 'change'), 2)

# for(i in 1:ncol(com)) {
#     a <- com[, i][1]
#     b <- com[, i][2]
#     r <- rmcorr(participant = 'pcode', measure1=a, measure2=b, dataset=data)
#     cat(a, '-', b, ': R =', r$r, '(p =', r$p, ') \n')
# }

## Sensor Data

In [14]:
import os
import pandas as pd
from typing import Optional


def _load_data(
    name: str
) -> Optional[pd.DataFrame]:
    paths = [
        (d, os.path.join(PATH_SENSOR, d, f'{name}.csv'))
        for d in os.listdir(PATH_SENSOR)
        if d.startswith('P')
    ]
    return pd.concat(
        filter(
            lambda x: len(x.index), 
            [
                pd.read_csv(p).assign(pcode=pcode)
                for pcode, p in paths
                if os.path.exists(p)
            ]
        ), ignore_index=True
    ).assign(
        timestamp=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=True).dt.tz_convert(DEFAULT_TZ)
    ).set_index(
        ['pcode', 'timestamp']
    )

In [15]:
import pandas as pd
import gc
from datetime import timedelta as td


# STATS = []

# for data_type in DATA_TYPES:
#     dat = _load_data(data_type)
#     inst = dat.groupby('pcode').count().iloc[:, -1]
#     samp = np.concatenate([
#         (dat.loc[(p,), :].index.array - dat.loc[(p,), :].index.array.shift(1)).dropna().total_seconds()
#         for p in dat.index.get_level_values('pcode').unique()
#     ])
#     inst, samp = summary(inst), summary(samp)
    
#     print('#'*5, data_type, '#'*5)
#     print('- # Inst.:', inst)
#     print('- Samp. period:', samp)
    
#     for c in dat.columns:
#         print(f'- {c}:', summary(dat[c]))
        
#     del dat
#     gc.collect()
    
# STATS = pd.DataFrame(STATS)

# Preprocessing

## Label

Because we intended to collected participants' responses to ESMs not voluntary responses, we screend out some responses as follows:
* We first screen out ESM responses that does not have 'scheduledTime' (meaning that a given ESM was expired or participants voluntarily reported their affective states regardless of ESM delivery). 
* Since we will evaluate our model using LOSO, the small number of responses for each participant might lead to inappropriate performance evaluation. We emprically set the number of the minimum responses upon ESM delivery as 5 per day (i.e., a half of our guides), so that we excluded participants whose responses to ESM less than 35.

In [16]:
LABELS_VALID = LABELS.loc[
    lambda x: ~x['scheduledTime'].isna(), :
]
# print(LABELS_VALID)
# print(f'# Non-voluntary response: {len(LABELS_VALID)}')
# print(summary(LABELS_VALID.groupby('pcode').count().iloc[:, -1]))

excl_pcode = LABELS_VALID.loc[
    lambda x: ~x['scheduledTime'].isna()
].groupby('pcode').count().iloc[:, -1].loc[lambda y: y < 35]

LABELS_VALID = LABELS_VALID.loc[
    lambda x:  ~x.index.get_level_values('pcode').isin(excl_pcode.index), :
]
print(f'# Response from participants with enough responses: {len(LABELS_VALID)}')
print(summary(LABELS_VALID.groupby('pcode').count().iloc[:, -1]))

print('# Participants whose responses to ESM delivery were less then 35')
# print(excl_pcode, f'#participants = {len(excl_pcode)} / #response = {sum(excl_pcode)}')
print(f'# participants = {len(excl_pcode)} / #response = {sum(excl_pcode)}')

# LABELS_VALID # 참여자: 47명, 응답 2619개
# 35개의 응답 보다 작은 참여자: 29명 (47/2619 -> 29/704)

# 연구는 47명으로 진행

# Response from participants with enough responses: 2619
{'n': 47, 'sum': 2619, 'mean': 55.723404255319146, 'SD': 13.076201628480542, 'med': 52.0, 'range': (36, 83), 'conf.': (51.88408762653044, 59.562720884107854), 'nan_count': 0}
# Participants whose responses to ESM delivery were less then 35
# participants = 29 / #response = 704


Here we consider binary classifications for valence, arousal, stress, and disturbance, in which a label value greater than 0 is "HIGH" (1) and the rest is "LOW" (0), at the arrival of ESM prompts (*scheduledTime*)

In [17]:
import pandas as pd
import numpy as np

LABELS_PROC = (
    LABELS_VALID
    .reset_index()
    .assign(
        timestamp=lambda x: pd.to_datetime(x['scheduledTime'], unit='ms', utc=True).dt.tz_convert(DEFAULT_TZ),
        attention_bin=lambda x: np.where(x['attention'] > 0, 1, 0)
    )
    .loc[:, ['pcode', 'timestamp', 'attention', 'attention_bin']]  # attention: 연속형, attention_bin: 이진분류된 데이터
    .set_index(['pcode', 'timestamp'])
)

LABELS_PROC

attention  attention_bin
pcode timestamp                                          
P01   2019-05-08 10:26:00+09:00          3              1
      2019-05-08 11:13:00+09:00          2              1
      2019-05-08 15:56:00+09:00          3              1
      2019-05-08 16:41:00+09:00          3              1
      2019-05-08 17:23:00+09:00          3              1
...                                    ...            ...
P80   2019-05-05 21:57:00+09:00         -3              0
      2019-05-06 15:06:00+09:00         -2              0
      2019-05-06 15:53:00+09:00          3              1
      2019-05-06 19:39:00+09:00         -1              0
      2019-05-06 21:08:00+09:00          3              1

[2619 rows x 2 columns]

In [18]:
from datetime import timedelta

# 10분 범위 설정
TIME_DELTA = timedelta(minutes=10)

# 결과를 저장할 리스트
rows = []

# 각 pcode별로 반복
for pcode, group in LABELS_PROC.groupby(level='pcode'):
    group = group.reset_index()

    # 각 attention_bin이 있는 행에 대해
    for _, row in group.iterrows():
        ts = row['timestamp']
        bin_val = row['attention_bin']

        # ±10분 범위 내 timestamp 생성
        timestamps = pd.date_range(ts - TIME_DELTA, ts + TIME_DELTA, freq='T', tz=DEFAULT_TZ)

        for t in timestamps:
            rows.append((pcode, t, bin_val))

# 확장된 데이터프레임 생성
LABELS_PROC = pd.DataFrame(rows, columns=['pcode', 'timestamp', 'attention_bin'])

# 중복 제거 및 인덱스 정렬
LABELS_PROC = (
    LABELS_PROC
    .drop_duplicates(subset=['pcode', 'timestamp'], keep='last')
    .set_index(['pcode', 'timestamp'])
    .sort_index()
)

# 확인 출력
print(LABELS_PROC.head(10))

# 이후 코드에 변수명 통일
inst = LABELS_PROC.groupby('pcode').count().iloc[:, -1]

for c in [c for c in LABELS_PROC.columns if c.endswith('_bin')]:
    print(f'- {c}:', summary(LABELS_PROC[c].astype(object)))

                                 attention_bin
pcode timestamp                               
P01   2019-05-08 10:16:00+09:00              1
      2019-05-08 10:17:00+09:00              1
      2019-05-08 10:18:00+09:00              1
      2019-05-08 10:19:00+09:00              1
      2019-05-08 10:20:00+09:00              1
      2019-05-08 10:21:00+09:00              1
      2019-05-08 10:22:00+09:00              1
      2019-05-08 10:23:00+09:00              1
      2019-05-08 10:24:00+09:00              1
      2019-05-08 10:25:00+09:00              1
- attention_bin: {'n': 54999, 'cardinality': 2, 'value_count': '0:27552, 1:27447'}


In [19]:
import numpy as np


inst = LABELS_PROC.groupby('pcode').count().iloc[:, -1]
for c in [c for c in LABELS_PROC.columns if c.endswith('_bin')]:
    print(f'- {c}:', summary(LABELS_PROC[c].astype(object)))

# 총 응답으로 볼 때, attention의 0, 1 클래스는 balanced
# 데이터셋으르 합칠 경우, oversampling 필요 없을 듯.

- attention_bin: {'n': 54999, 'cardinality': 2, 'value_count': '0:27552, 1:27447'}


- 1. attention_bin 의 샘플 수를 기준으로 분할?
- 2. feature extraction 이후 최종 나온 feature 정보들을 분할?

In [20]:
import pandas as pd

inst = LABELS_PROC.groupby('pcode').count().iloc[:, -1]

for c in [col for col in LABELS_PROC.columns if col.endswith('_bin')]:
    counts = LABELS_PROC.groupby('pcode')[c].value_counts().unstack(fill_value=0)
    ratios = counts.div(counts.sum(axis=1), axis=0)
    stats = pd.DataFrame({
        'n_total': counts.sum(axis=1),
        'n_class_0': counts.get(0, 0),
        'n_class_1': counts.get(1, 0),
        'ratio_0': ratios.get(0, 0),
        'ratio_1': ratios.get(1, 0),
        'imbalance_ratio': counts.max(axis=1) / counts.sum(axis=1)
    })
    print("클래스별 평균 및 표준편차:")
    for col in ['ratio_0', 'ratio_1', 'imbalance_ratio']:
        mean = stats[col].mean()
        std = stats[col].std()
        print(f"  - {col}: 평균 = {mean:.3f}, 표준편차 = {std:.3f}")

# imbalance_ratio: 각 참가자의 샘플의 다수 클래스 비율
# 값이 0.5에 가까우면 균형된 데이터
# 문제: 전체 데이터에서는 균형, 개별 참가자 단위로는 불균형 -> LOGO시 문제 발생 가능성.
# 해결: class_weight='balanced', oversampling, 불균형 참가자 제거, Leave-Multiple-Groups-Out
# 평가지표? F1, ROC-AUC가 불균형에 민감한지?

클래스별 평균 및 표준편차:
  - ratio_0: 평균 = 0.500, 표준편차 = 0.206
  - ratio_1: 평균 = 0.500, 표준편차 = 0.206
  - imbalance_ratio: 평균 = 0.665, 표준편차 = 0.121


## Sensor Data

For each type of sensor data, we applied different preprocessing. Detailed decsription is provided in the paper.



### Implementation

In [21]:
import pandas as pd
import numpy as np
from typing import Dict, Union

# IQR -> 이상치 제거
def _remove_outliers_iqr(series: pd.Series) -> pd.Series:
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series.clip(lower=lower, upper=upper)

# 로그 정규화 + Z-score
def _log_normalize(series: pd.Series) -> pd.Series:
    series = series.clip(lower=1)
    log_vals = np.log1p(series)
    return (log_vals - log_vals.mean()) / log_vals.std()

# SkinTemperature.csv
def _proc_skin_temperature(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    temp = data['temperature'].astype('float32')
    temp = _remove_outliers_iqr(temp)
    return (temp - temp.mean()) / temp.std()

# RRI.csv
def _proc_rri(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    rri = data['interval'].astype('float32')
    rri = _remove_outliers_iqr(rri)
    return (rri - rri.mean()) / rri.std()

# HR.csv
def _proc_hr(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    hr = data['bpm'].astype('float32')
    hr = _remove_outliers_iqr(hr)
    return (hr - hr.mean()) / hr.std()

# EDA.csv
def _proc_eda(data: pd.DataFrame) -> Union[pd.Series, Dict[str, pd.Series]]:
    eda = data['resistance'].astype('float32')
    eda = _remove_outliers_iqr(eda)
    return _log_normalize(eda)

### Execution

In [22]:
import pandas as pd
import gc
from functools import reduce
import time

## 최종 모델에서는 4개 feature 전부 다 사용하면 될 듯

FUNC_PROC = {
    # 'EDA': _proc_eda,
    'HR': _proc_hr,
    # 'RRI': _proc_rri,
    # 'SkinTemperature': _proc_skin_temperature,
}


def _process(data_type: str):
    log(f'Begin to processing data: {data_type}')
    
    abbrev = DATA_TYPES[data_type]
    data_raw = _load_data(data_type)
    data_proc = FUNC_PROC[data_type](data_raw)
    result = dict()
    
    if type(data_proc) is dict:
        for k, v in data_proc.items():
            result[f'{abbrev}_{k}'] = v
    else:
        result[abbrev] = data_proc
        
    log(f'Complete processing data: {data_type}')
    return result


with on_ray(num_cpus=12):
    jobs = []
    
    func = ray.remote(_process).remote
    
    for data_type in DATA_TYPES:
        job = func(data_type)
        jobs.append(job)

    jobs = ray.get(jobs)
    
    # 메모리 최적화를 위해 추가 
    combined_result = {}
    for d in jobs:
        combined_result |= d
    
    t0 = time.time()
    dump(combined_result, os.path.join(PATH_INTERMEDIATE, 'proc.pkl'))
    log(f'[SAVE] done in {time.time() - t0:.1f}s')
    
    del jobs
    gc.collect()

In [23]:
import os
import gc

# DATA_ITEMS:
# [('EDA', pcode  timestamp                       
# P19    2019-05-08 09:00:00.029000+09:00     10070.0
#        2019-05-08 09:00:00.233000+09:00     10010.0
#        2019-05-08 09:00:00.434000+09:00     10160.0
#        2019-05-08 09:00:00.640000+09:00     10130.0
#        2019-05-08 09:00:00.842000+09:00     10130.0

def _remove_outliers_iqr_np(arr: np.ndarray) -> np.ndarray: # IQR 방식으로 샘플링 주기 이상치 제거
    q1 = np.percentile(arr, 25)
    q3 = np.percentile(arr, 75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return arr[(arr >= lower) & (arr <= upper)]


DATA = load(os.path.join(PATH_INTERMEDIATE, 'proc.pkl'))
#categorial, numeric 변수 수 계산
N_NUMERIC, N_CATEGORICAL = 0, 0

for k, v in DATA.items():
    if v.dtype.kind.isupper() or v.dtype.kind == 'b': 
        N_CATEGORICAL += 1
    else:
        N_NUMERIC += 1

    inst = v.groupby('pcode').count()
    
    # 샘플링 주기 계산 및 이상치 제거
    sam = np.concatenate([
        (v.loc[(p,)].index.array - v.loc[(p,)].index.array.shift(1)).dropna().total_seconds()
        for p in v.index.get_level_values('pcode').unique()
    ])
    sam = _remove_outliers_iqr_np(sam)

    
    print('#'*5, k, '#'*5, )
#     print('- # Inst.:', inst)
    print('- Samp. period:', summary(sam))
    print('- Values', summary(v))
    print('')
    
    
print(f'# categorical data: {N_CATEGORICAL}/# numeric data: {N_NUMERIC}')
del DATA
gc.collect()

# IQR 방식으로 샘플링 주기 이상치 제거 전
# | 센서   | 샘플링 주기 평균 (s)     | 샘플링 주기 SD    
# |-------|----------------------|----------------
# | EDA   | 0.514                | 140.55         
# | HRT   | 3.008                | 362.92         
# | RRI   | 1.985                | 276.20         
# | SKT   | 77.03                | 1,719.19       

# IQR 방식으로 샘플링 주기 이상치 제거 후
# | 센서   | 샘플링 주기 평균 (s)     | 샘플링 주기 SD    
# |-------|----------------------|----------------
# | EDA   | 0.199                | 0.013          
# | HRT   | 0.996                | 0.014          
# | RRI   | 0.761                | 0.172          
# | SKT   | 30.08                | 0.014          

# - EDA:
#   · 많이 이상함. 전처리 필요
#   · 매우 빠른 샘플링(0.5초), 샘플링 주기의 표준편차가 매우 큼. 
#   · 샘플 누락, 중단, 오류 구간의 존재 가능성?, 극단적으로 큰 샘플링 주기가 존재?
#     -> 이상치 제거, 리샘플링, (스케일이 너무 큼 -> 정규화(로그 변환, 정규화))
#   · 측정값이 큼 -> log 변환 고려?

# - HRT:
#   · 평균 주기 3초, SD=362초 -> 이것도 이상함, 이상치 제거 필요?
#   · 평균, 표준편차 왜곡?
#

##### HRT #####
- Samp. period: {'n': 12126885, 'sum': 12080570.163000021, 'mean': 0.9961808133745823, 'SD': 0.01388122295369463, 'med': 0.996, 'range': (0.956, 1.036), 'conf.': (0.9961730006730288, 0.9961886260761358), 'nan_count': 0}
- Values {'n': 13621023, 'sum': 15.389024166737158, 'mean': 1.129799440668822e-06, 'SD': 0.8123838625366665, 'med': -0.028077235982838025, 'range': (-1.8874378396168132, 1.8312833676511373), 'conf.': (-0.0004302944520892503, 0.00043255405097058797), 'nan_count': 0}

# categorical data: 0/# numeric data: 1


0

# Feature Extraction

## Implementation

In [24]:
import numpy as np
import pandas as pd
from typing import Dict, Callable, Union, Tuple, List, Optional, Iterable
from datetime import timedelta as td
from scipy import stats
from scipy.interpolate import CubicSpline
import ray
import warnings
import time

def _safe_na_check(_v):
    _is_nan_inf = False
    try:
        _is_nan_inf = np.isnan(_v) or np.isinf(_v)
    except:
        _is_nan_inf = False
    return _is_nan_inf or _v is None

# _extract: 한 명의 참가자(pid)의 feature 생성
# _extract 출력:
# X: DataFrame(n, f) / n = 라벨 개수, f = 추출된 feature 수 / row: timestamp에 대한 feature vector, col: 센서, 시간 기반 피쳐
# y: np.ndarray(n,) / timestamp별 라벨(1D)
# group: np.ndarray(n,) / 참가자 ID 반복 배열 (모델 그룹 분리용) / 각 row 별 pid - KFold, LOGO에 사용 / 입력 파라미터의 pid와 동일값 반복
# date_times: np.ndarray(n,) / 각 sample의 timestamp
def _extract(
        pid: str,
        data: Dict[str, pd.Series], # pcode, timestamp으로 구성된 센서의 시계열 데이터
        label: pd.Series, # timestamp별 라벨
        label_values: List[str], # 가능한 class: [0, 1]
        window_data: Dict[str, Union[int, Callable[[pd.Timestamp], int]]], # 센서별 time-window 크기
        window_label: Dict[str, Union[int, Callable[[pd.Timestamp], int]]], # label의 time-window 크기
        categories: Dict[str, Optional[List[any]]] = None, # 범주형 센서
        resample_s: Dict[str, float] = None # 센서별 리샘플링 간격
) -> Tuple[pd.DataFrame, np.ndarray, np.ndarray, np.ndarray]:
    _s = time.time()
    log(f"Begin feature extraction on {pid}'s data.")

    categories = categories or dict()
    resample_s = resample_s or dict()

    X, y, date_times = [], [], [] # 상단 주석 참고
 
    for timestamp in label.index:
        row = dict() # 각 timestamp 별 feature을 저장

        label_cur = label.at[timestamp]
        t = timestamp - td(milliseconds=1)

        # Features from sensor data
        for d_key, d_val in data.items():
            is_numeric = d_key not in categories
            cats = categories.get(d_key) or list()
            d_val = d_val.sort_index()

            if is_numeric or cats:
                try:
                    v = d_val.loc[:t].iloc[-1]             # t 시점 직전 가장 마지막 센서 값을 사용
                except (KeyError, IndexError):
                    v = 0

                if is_numeric:
                    row[f'{d_key}#VAL'] = v
                else:
                    for c in cats:
                        row[f'{d_key}#VAL={c}'] = v == c

            # catogorial 데이터의 최근 상태 변화 시간
            if not is_numeric:
                try:
                    v = d_val.loc[:t]
                    row[f'{d_key}#DSC'] = (t - v.index[-1]).total_seconds() if len(v) else -1.0
                    for c in cats:
                        v_sub = v.loc[lambda x: x == c].index
                        row[f'{d_key}#DSC={c}'] = (t - v_sub[-1]).total_seconds() if len(v_sub) else -1.0
                except (KeyError, IndexError):
                    row[f'{d_key}#DSC'] = -1.0
                    for c in cats:
                        row[f'{d_key}#DSC={c}'] = -1.0

            # Time-window 기반 피처 (resampling 포함)
            sample_rate = resample_s.get(d_key) or 1
            d_val_res = d_val.resample(f'{sample_rate}s', origin='start')
            if is_numeric:
                """ 보간 방식 추가 부분 """
                if d_key == "interval":
                    interval_series = d_val.dropna()
                    if len(interval_series) >= 4:
                        ts = interval_series.index.view(np.int64) // 10**6
                        cs = CubicSpline(ts, interval_series.values)
                        full_ts = d_val.index.view(np.int64) // 10**6
                        d_val[:] = cs(full_ts)
                else: 
                    d_val_res = d_val_res.mean().interpolate(method='linear').dropna()
            else:
                d_val_res = d_val_res.ffill().dropna()

            for w_key, w_val in window_data.items():
                w_val = w_val(t) if isinstance(w_val, Callable) else w_val
                try:
                    v = d_val_res.loc[t - td(seconds=w_val):t]
                    # numeric 데이터일 경우에만 변환
                    if is_numeric:
                        v_arr = v.values.astype(np.float64)
                    else:
                        v_arr = v
                except (KeyError, IndexError):
                    continue

                with warnings.catch_warnings():
                    warnings.simplefilter('ignore')

                    if is_numeric:
                        hist, _ = np.histogram(v, bins='doane', density=False)
                        std = np.sqrt(np.var(v, ddof=1)) if len(v) > 1 else 0
                        v_norm = (v - np.mean(v)) / std if std != 0 else np.zeros(len(v))
                        # window_data 기준으로 구간 추출 - 통계 기반
                        row[f'{d_key}#AVG#{w_key}'] = np.float32(np.mean(v_arr))
                        row[f'{d_key}#STD#{w_key}'] = np.float32(std)
                        row[f'{d_key}#SKW#{w_key}'] = np.float32(stats.skew(v_arr, bias=False))
                        row[f'{d_key}#KUR#{w_key}'] = np.float32(stats.kurtosis(v_arr, bias=False))
                        row[f'{d_key}#ASC#{w_key}'] = np.float32(np.sum(np.abs(np.diff(v_arr))))
                        row[f'{d_key}#BEP#{w_key}'] = np.float32(stats.entropy(hist))
                        row[f'{d_key}#MED#{w_key}'] = np.float32(np.median(v_arr))
                        # TSC: 시계열 복잡성. √(Δ값 제곱합) 
                        row[f'{d_key}#TSC#{w_key}'] = np.float32(np.sqrt(np.sum(np.power(np.diff(v_norm), 2))))
                    else:
                        cnt = v.value_counts()
                        val, sup = cnt.index, cnt.values
                        hist = {k: v for k, v in zip(val, sup)}

                        row[f'{d_key}#ETP#{w_key}'] = stats.entropy(sup / len(v))
                        row[f'{d_key}#ASC#{w_key}'] = np.sum(v.values[1:] != v.values[:-1])

                        if len(cats) == 2:
                            c = cats[0]
                            row[f'{d_key}#DUR#{w_key}'] = hist[c] / len(v) if c in hist else 0
                        else:
                            for c in cats:
                                row[f'{d_key}#DUR={c}#{w_key}'] = hist[c] / len(v) if c in hist else 0

        # 시간 기반 피처: one-hot encoding
        day_of_week = ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN'][t.isoweekday() - 1]
        is_weekend = 'Y' if t.isoweekday() > 5 else 'N'
        hour = t.hour

        # 시간대별 패턴: 비정기적 설문조사 주기에 맞게 하루를 7개의 시간대로 분할
        if 6 <= hour < 9:
            hour_name = 'DAWN'
        elif 9 <= hour < 12:
            hour_name = 'MORNING'
        elif 12 <= hour < 15:
            hour_name = 'AFTERNOON'
        elif 15 <= hour < 18:
            hour_name = 'LATE_AFTERNOON'
        elif 18 <= hour < 21:
            hour_name = 'EVENING'
        elif 21 <= hour < 24:
            hour_name = 'NIGHT'
        else:
            hour_name = 'MIDNIGHT'

        for d in ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN']:
            row[f'ESM#DOW={d}'] = d == day_of_week

        for d in ['Y', 'N']:
            row[f'ESM#WKD={d}'] = d == is_weekend

        for d in ['DAWN', 'MORNING', 'AFTERNOON', 'LATE_AFTERNOON', 'EVENING', 'NIGHT', 'MIDNIGHT']:
            row[f'ESM#HRN={d}'] = d == hour_name

        # 응답 이력 기반 피처: 지난 3시간 내 (예시) 응답 중 긍정/부정 응답 비율
        for w_key, w_val in window_label.items():
            w_val = w_val(t) if isinstance(w_val, Callable) else w_val
            try:
                v = label.loc[t - td(seconds=w_val):t]
                if len(label_values) <= 2:
                    row[f'ESM#LIK#{w_key}'] = np.sum(v == label_values[0]) / len(v) if len(v) > 0 else 0
                else:
                    for l in label_values:
                        row[f'ESM#LIK={l}#{w_key}'] = np.sum(v == l) / len(v) if len(v) > 0 else 0
            except (KeyError, IndexError):
                if len(label_values) <= 2: 
                    row[f'ESM#LIK#{w_key}'] = 0
                else:
                    for l in label_values:
                        row[f'ESM#LIK={l}#{w_key}'] = 0

        row = {
            k: 0.0 if _safe_na_check(v) else v
            for k, v in row.items()
        }
        X.append(row)
        y.append(label_cur)
        date_times.append(timestamp)

    log(f"Complete feature extraction on {pid}'s data ({time.time() - _s:.2f} s).")
    X, y, group, date_times = pd.DataFrame(X), np.asarray(y), np.repeat(pid, len(y)), np.asarray(date_times)
    
    #출력 결과 코드
    print(f"[{pid}] Extracted {X.shape[0]} samples with {X.shape[1]} features")
    return X, y, group, date_times

# _extract를 참가자 리스트 전체에 병렬/순차적으로 적용 -> 최종 feature 데이터
# 출력: 
# X: 전체 참가자의 feature 데이터 (DataFrame)
# y: 전체 라벨
# group: 각 행에 해당하는 참가자의 ID
# t_norm: timestamp를 기준 시점으로부터 정규화한 시간 (초 단위), XGB, NN에 사용 가능
# date_times: 각 sample의 실제 timestamp (Datetime array)

def extract(
        pids: Iterable[str], 
        data: Dict[str, pd.Series], # 전체 센서 데이터
        label: pd.Series, # 전체 라벨
        label_values: List[str],
        window_data: Dict[str, Union[int, Callable[[pd.Timestamp], int]]],
        window_label: Dict[str, Union[int, Callable[[pd.Timestamp], int]]],        
        categories: Dict[str, Optional[List[any]]] = None,        
        resample_s: Dict[str, float] = None,
        with_ray: bool=False
):
    if with_ray and not ray.is_initialized():
        raise EnvironmentError('Ray should be initialized if "with_ray" is set as True.')

    func = ray.remote(_extract).remote if with_ray else _extract
    jobs = []
    
    # 참가자별로 _extract 실행 (단일 or 병렬)
    for pid in pids:
        d = dict()
        for k, v in data.items():
            try:
                d[k] = v.loc[(pid, )]
            except (KeyError, IndexError):
                pass

        job = func(
            pid=pid,
            data=d,
            label=label.loc[(pid, )],
            label_values=label_values,
            window_data=window_data,
            window_label=window_label,
            categories=categories,
            resample_s=resample_s
        )
        jobs.append(job)

    jobs = ray.get(jobs) if with_ray else jobs

    X = pd.concat([x for x, _, _, _ in jobs], axis=0, ignore_index=True)
    y = np.concatenate([x for _, x, _, _ in jobs], axis=0)
    group = np.concatenate([x for _, _, x, _ in jobs], axis=0)
    date_times = np.concatenate([x for _, _, _, x in jobs], axis=0)
    
    # 모든 참가자의 feature / label 결과 통합. timestamp를 자정 기준으로 정렬 -> t_norm(상대적 시간) 계산 위한 준비
    t_s = date_times.min().normalize().timestamp()
    # datetime을 수치화된 상대 시간으로 변환. 머신러닝의 효과적인 학습 위함. t_s(시작 시간)기준으로 얼마나 지났는지
    t_norm = np.asarray(list(map(lambda x: x.timestamp() - t_s, date_times)))

    # 결측치 처리 및 dtype 설정
    C, DTYPE = X.columns, X.dtypes

    X = X.fillna({
        **{c: False for c in C[(DTYPE == object) | (DTYPE == bool)]},
        **{c: 0.0 for c in C[(DTYPE != object) & (DTYPE != bool)]},
    }).astype({
        **{c: 'bool' for c in C[(DTYPE == object) | (DTYPE == bool)]},
        **{c: 'float32' for c in C[(DTYPE != object) & (DTYPE != bool)]},
    })

    return X, y, group, t_norm, date_times

## Execution

In [25]:
LABEL_VALUES = [1, 0]

WINDOW_DATA = {
    'S30': 30,
    'M01': 60,
    'M05': 60 * 5,
    'M10': 60 * 10,
    'M30': 60 * 30,
    'H01': 60 * 60,
    # 'H02': 60 * 60 * 2, # 추가 
    'H03': 60 * 60 * 3,
    # 'H04': 60 * 60 * 4, # 추가 
    # 'H05': 60 * 60 * 5, # 추가
    'H06': 60 * 60 * 6
}

WINDOW_LABEL = {
    'H06': 60 * 60 * 6,
    'H12': 60 * 60 * 12,
    'H24': 60 * 60 * 24,
}

RESAMPLE_s = {
    'EDA': 0.25,
}

DATA = load(os.path.join(PATH_INTERMEDIATE, 'proc.pkl'))

In [26]:
with on_ray(num_cpus=12):
    l = 'attention'

    labels = LABELS_PROC[f'{l}_bin']
    pids = labels.index.get_level_values('pcode').unique()

    feat = extract(
        pids=pids, 
        data=DATA,         
        label=labels,
        label_values=LABEL_VALUES,
        window_data=WINDOW_DATA,
        window_label=WINDOW_LABEL,
        resample_s=RESAMPLE_s,
        with_ray=True
    )

    dump(feat, os.path.join(PATH_INTERMEDIATE, f'{l}.pkl'))

# 결과 timestamp 기반 응답 수가 35~83 -> 참가자 간 f1-score의 분산이 큰 경우, 조정 필요
# Extracted 54 samples with 279 features: 응답수 54개, feature 279개

In [27]:
import numpy as np
import os

# attention만 대상
X, y, group, t, _ = load(os.path.join(PATH_INTERMEDIATE, 'attention.pkl'))

print(f'# attention')
# categorical feature은 시간 기반 정보에서 생성. ESM 응답 이력 + 시간대
print(f'- Feature space: {len(X.dtypes)}; Cat.: {np.sum(X.dtypes == bool)}; Num.: {np.sum(X.dtypes != bool)}')
print(f'- Label distribution: {np.unique(y, return_counts=True)}')

print("# attention feature extraction summary")
print(f"- Feature matrix: X.shape = {X.shape}  (rows: {X.shape[0]}, features: {X.shape[1]})")
print(f"- Label vector:   y.shape = {y.shape}")
print(f"- Group vector:   group.shape = {group.shape}")
print(f"- Time norm:      t.shape = {t.shape}")

print(f"- Label distribution: {dict(zip(*np.unique(y, return_counts=True)))}")
print(f"- Feature types: Cat. = {np.sum(X.dtypes == bool)}, Num. = {np.sum(X.dtypes != bool)}")

# attention
- Feature space: 84; Cat.: 16; Num.: 68
- Label distribution: (array([0, 1], dtype=int64), array([27552, 27447], dtype=int64))
# attention feature extraction summary
- Feature matrix: X.shape = (54999, 84)  (rows: 54999, features: 84)
- Label vector:   y.shape = (54999,)
- Group vector:   group.shape = (54999,)
- Time norm:      t.shape = (54999,)
- Label distribution: {0: 27552, 1: 27447}
- Feature types: Cat. = 16, Num. = 68


In [70]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def split_data():
    # attention 데이터 로드
    X, y, group, t, _ = load(os.path.join('intermediate', 'attention.pkl'))

    print(f'# attention')
    print(f'- Feature space: {len(X.dtypes)}; Cat.: {np.sum(X.dtypes == bool)}; Num.: {np.sum(X.dtypes != bool)}')
    print(f'- Label distribution: {np.unique(y, return_counts=True)}')

    # 1. 먼저 train+val과 test로 분할 (90:10)
    X_train_val, X_test, y_train_val, y_test, group_train_val, group_test = train_test_split(
        X, y, group,
        test_size=0.1,
        stratify=y,
        random_state=42
    )

    # 2. train_val을 다시 train과 validation으로 분할 (80:10)
    X_train, X_val, y_train, y_val, group_train, group_val = train_test_split(
        X_train_val, y_train_val, group_train_val,
        test_size=0.111,  # 0.1/0.9 ≈ 0.111
        stratify=y_train_val,
        random_state=42
    )

    print("\n# Data Split Summary")
    print(f"Train set: {X_train.shape[0]} samples")
    print(f"Validation set: {X_val.shape[0]} samples")
    print(f"Test set(simulation 용): {X_test.shape[0]} samples")

    print("\n# Label Distribution in Each Set")
    print("Train:", dict(zip(*np.unique(y_train, return_counts=True))))
    print("Validation:", dict(zip(*np.unique(y_val, return_counts=True))))
    print("Test(simulation용):", dict(zip(*np.unique(y_test, return_counts=True))))

    # 각 세트별로 시간 정보 추출
    train_val_indices = np.where(np.isin(group, group_train_val))[0]
    test_indices = np.where(np.isin(group, group_test))[0]

    # train+val 데이터와 test 데이터 저장 (원본과 동일한 구조 유지)
    train_val_data = (X_train_val, y_train_val, group_train_val, t[train_val_indices], None)
    test_data = (X_test, y_test, group_test, t[test_indices], None)

    # 각각의 파일로 저장
    dump(train_val_data, os.path.join('intermediate', 'attention_train_val.pkl'))
    dump(test_data, os.path.join('intermediate', 'attention_test.pkl'))

    print("\n# Saved files:")
    print("- attention_train_val.pkl (train + validation data)")
    print("- attention_test.pkl (test data)")

In [71]:
split_data()

# attention
- Feature space: 84; Cat.: 16; Num.: 68
- Label distribution: (array([0, 1], dtype=int64), array([27552, 27447], dtype=int64))

# Data Split Summary
Train set: 44004 samples
Validation set: 5495 samples
Test set(simulation 용): 5500 samples

# Label Distribution in Each Set
Train: {0: 22044, 1: 21960}
Validation: {0: 2753, 1: 2742}
Test(simulation용): {0: 2755, 1: 2745}

# Saved files:
- attention_train_val.pkl (train + validation data)
- attention_test.pkl (test data)


In [74]:
import numpy as np
import os

# attention만 대상
X, y, group, t, _ = load(os.path.join(PATH_INTERMEDIATE, 'attention_train_val.pkl'))

print(f'# attention_train_val (Train + Val : "학습 & 모델링")')
# categorical feature은 시간 기반 정보에서 생성. ESM 응답 이력 + 시간대
print(f'- Feature space: {len(X.dtypes)}; Cat.: {np.sum(X.dtypes == bool)}; Num.: {np.sum(X.dtypes != bool)}')
print(f'- Label distribution: {np.unique(y, return_counts=True)} \n')

print("# attention feature extraction summary")
print(f"- Feature matrix: X.shape = {X.shape}  (rows: {X.shape[0]}, features: {X.shape[1]})")
print(f"- Label vector:   y.shape = {y.shape}")
print(f"- Group vector:   group.shape = {group.shape}")
print(f"- Time norm:      t.shape = {t.shape}")

print(f"- Label distribution: {dict(zip(*np.unique(y, return_counts=True)))}")
print(f"- Feature types: Cat. = {np.sum(X.dtypes == bool)}, Num. = {np.sum(X.dtypes != bool)}")

# attention_train_val (Train + Val : "학습 & 모델링")
- Feature space: 84; Cat.: 16; Num.: 68
- Label distribution: (array([0, 1], dtype=int64), array([24797, 24702], dtype=int64)) 

# attention feature extraction summary
- Feature matrix: X.shape = (49499, 84)  (rows: 49499, features: 84)
- Label vector:   y.shape = (49499,)
- Group vector:   group.shape = (49499,)
- Time norm:      t.shape = (54999,)
- Label distribution: {0: 24797, 1: 24702}
- Feature types: Cat. = 16, Num. = 68


In [75]:
import numpy as np
import os

# attention만 대상
X, y, group, t, _ = load(os.path.join(PATH_INTERMEDIATE, 'attention_test.pkl'))

print(f'# attention_test (test : "offline 시뮬레이션")')
# categorical feature은 시간 기반 정보에서 생성. ESM 응답 이력 + 시간대
print(f'- Feature space: {len(X.dtypes)}; Cat.: {np.sum(X.dtypes == bool)}; Num.: {np.sum(X.dtypes != bool)}')
print(f'- Label distribution: {np.unique(y, return_counts=True)} \n')

print("# attention feature extraction summary")
print(f"- Feature matrix: X.shape = {X.shape}  (rows: {X.shape[0]}, features: {X.shape[1]})")
print(f"- Label vector:   y.shape = {y.shape}")
print(f"- Group vector:   group.shape = {group.shape}")
print(f"- Time norm:      t.shape = {t.shape}")

print(f"- Label distribution: {dict(zip(*np.unique(y, return_counts=True)))}")
print(f"- Feature types: Cat. = {np.sum(X.dtypes == bool)}, Num. = {np.sum(X.dtypes != bool)}")

# attention_test (test : "offline 시뮬레이션")
- Feature space: 84; Cat.: 16; Num.: 68
- Label distribution: (array([0, 1], dtype=int64), array([2755, 2745], dtype=int64)) 

# attention feature extraction summary
- Feature matrix: X.shape = (5500, 84)  (rows: 5500, features: 84)
- Label vector:   y.shape = (5500,)
- Group vector:   group.shape = (5500,)
- Time norm:      t.shape = (54999,)
- Label distribution: {0: 2755, 1: 2745}
- Feature types: Cat. = 16, Num. = 68


Let's check whether the number of features is same as intented.

In [30]:
N_NUM, N_CAT_B, N_CAT_NB = 0, 0, 0 

for k, v in DATA.items():
    N_NUM = N_NUM + 1

# Features relavant to delivery time
N_TIM = 7 + 2 + 7
print(f'N_TIM: {N_TIM}')
        
# Features relevant to latest value
N_VAL_NUM = N_NUM
print(f'N_VAL_NUM: {N_VAL_NUM}')

# Features from time-windows
N_WIN_NUM = N_NUM * 8 * len(WINDOW_DATA)

print(f'N_WIN_NUM: {N_WIN_NUM}')


# Features from previous labels
N_LBL = len(WINDOW_LABEL) * (1 if len(LABEL_VALUES) <= 2 else len(LABEL_VALUES))
print(f'N_LBL: {N_LBL}')

N_FEAT = N_TIM + N_WIN_NUM + N_VAL_NUM + N_LBL
print(f'N_FEAT: {N_FEAT}')


N_TIM: 16
N_VAL_NUM: 1
N_WIN_NUM: 64
N_LBL: 3
N_FEAT: 84


Okay, features are extracted as intended.

# Cross-validation

## Implementation

In [31]:
# pip install imbalanced-learn

### CV Pipeline

In [55]:
import os
import pandas as pd
import numpy as np
import traceback as tb
from contextlib import contextmanager
from typing import Tuple, Dict, Union, Generator, List
from dataclasses import dataclass
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.base import BaseEstimator, clone
from sklearn.feature_selection import SelectFromModel
#stratifiedgroupkfold 추가
from sklearn.model_selection import StratifiedKFold, LeaveOneGroupOut, StratifiedShuffleSplit, RepeatedStratifiedKFold, StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
import time
import ray


@dataclass
class FoldResult:
    name: str
    estimator: BaseEstimator
    X_train: pd.DataFrame
    y_train: np.ndarray
    X_test: pd.DataFrame
    y_test: np.ndarray
    categories: Dict[str, Dict[int, str]] = None


# logo의 단점: 참가자별 attention label의 불균형을 해결하기 위한 stratification 불가
# StratifiedKFold 혹은 StratifiedGroupKFold 필요
# StratifiedKFold는 한 pcode의 데이터가 train-test 모두에 포함이 가능 -> StratifiedGroupKFold
def _split(
        alg: str,
        X: Union[pd.DataFrame, np.ndarray] = None,
        y: np.ndarray = None,
        groups: np.ndarray = None,
        random_state: int = None,
        n_splits: int = None,
        n_repeats: int = None,
        test_ratio: float = None
) -> Generator[Tuple[np.ndarray, np.ndarray], None, None]:
    if alg == 'holdout':
        splitter = StratifiedShuffleSplit(
            n_splits=n_splits,
            test_size=test_ratio,
            random_state=random_state
        )
    elif alg == 'kfold':
        if n_repeats and n_repeats > 1:
            splitter = RepeatedStratifiedKFold(
                n_splits=n_splits,
                n_repeats=n_repeats,
                random_state=random_state,
            )
        else:
            splitter = StratifiedKFold(
                n_splits=n_splits,
                random_state=random_state,
                shuffle=False if random_state is None else True,
            )
    elif alg == 'stratifiedgroupkfold':
        if n_repeats and n_repeats > 1:
            splitter = StratifiedGroupKFold(
                n_splits=n_splits,
                random_state=random_state,
            )
        else:
            splitter = StratifiedGroupKFold(
                n_splits=n_splits,
                random_state=random_state,
                shuffle=False if random_state is None else True,
            )
    elif alg == 'logo':
        splitter = LeaveOneGroupOut()
    else:
        raise ValueError('"alg" should be one of "holdout", "kfold", "logo", or "groupk".')

    split = splitter.split(X, y, groups)

    for I_train, I_test in split:
        yield I_train, I_test


def _train(
    dir_result: str,
    name: str,
    X_train: pd.DataFrame,
    y_train: np.ndarray,
    X_test: pd.DataFrame,
    y_test: np.ndarray,
    C_cat: np.ndarray,
    C_num: np.ndarray,
    estimator: BaseEstimator,
    normalize: bool = False,
    select: Union[List[SelectFromModel], SelectFromModel] = None,
    oversample: bool = False,
    random_state: int = None,
    categories: Union[List, Dict[str, Dict[int, str]]] = None
):
    @contextmanager
    def _log(task_type: str):
        log(f'In progress: {task_type}.')
        _t = time.time()
        _err = None
        _result = dict()
        
        try:
            yield _result
        except:
            _err = tb.format_exc()
        finally:
            _e = time.time() - _t
            if _err:
                _msg = f'Failure: {task_type} ({_e:.2f}s). Keep running without this task. Caused by: \n{_err}' 
            else:
                _msg = f'Success: {task_type} ({_e:.2f}s).' 
                if _result:
                    _r = '\n'.join([f'- {k}: {v}' for k, v in _result.items()])
                    _msg = f'{_msg}\n{_r}'
            log(_msg)
    
    if normalize:
        with _log(f'[{name}] Normalizing numeric features'):
            X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
            X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values
            
            scaler = StandardScaler().fit(X_train_N)
            X_train_N = scaler.transform(X_train_N)
            X_test_N = scaler.transform(X_test_N)
         
            X_train = pd.DataFrame(
                np.concatenate((X_train_C, X_train_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )
            X_test = pd.DataFrame(
                np.concatenate((X_test_C, X_test_N), axis=1),
                columns=np.concatenate((C_cat, C_num))
            )
           
    if select:
        if isinstance(select, SelectFromModel):
            select = [select]
            
        for i, s in enumerate(select):
            with _log(f'[{name}] {i+1}-th Feature selection') as r:
                C = np.asarray(X_train.columns)
                r['# Orig. Feat.'] = f'{len(C)} (# Cat. = {len(C_cat)}; # Num. = {len(C_num)})'
                M = s.fit(X=X_train.values, y=y_train).get_support()
                C_sel = C[M]
                C_cat = C_cat[np.isin(C_cat, C_sel)]
                C_num = C_num[np.isin(C_num, C_sel)]
                
                X_train_N, X_test_N = X_train[C_num].values, X_test[C_num].values
                X_train_C, X_test_C = X_train[C_cat].values, X_test[C_cat].values


                X_train = pd.DataFrame(
                    np.concatenate((X_train_C, X_train_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )
                X_test = pd.DataFrame(
                    np.concatenate((X_test_C, X_test_N), axis=1),
                    columns=np.concatenate((C_cat, C_num))
                )
                r['# Sel. Feat.'] = f'{len(C_sel)} (# Cat. = {len(C_cat)}; # Num. = {len(C_num)})'

    if oversample:
        with _log('Oversampling'):
            if len(C_cat):
                cat_mask = np.isin(X_train.columns, C_cat)
                cat_indices = np.where(cat_mask)[0]  # <- 인덱스 리스트로 변환
                sampler = SMOTENC(categorical_features=cat_indices.tolist(), random_state=random_state)
            else:
                sampler = SMOTE(random_state=random_state)
            X_train, y_train = sampler.fit_resample(X_train, y_train)

    with _log(f'[{name}] Training'):
        estimator = estimator.fit(X_train, y_train)
        result = FoldResult(
            name=name,
            estimator=estimator,
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            y_test=y_test,
            categories=categories
        )
        dump(result, os.path.join(dir_result, f'{name}.pkl'))
    

# def cross_val(
#     X: pd.DataFrame,
#     y: np.ndarray,
#     groups: np.ndarray,
#     path: str,
#     name: str,
#     estimator: BaseEstimator,
#     categories: List[str] = None,
#     normalize: bool = False,
#     split: str = None,
#     split_params: Dict[str, any] = None,
#     select: Union[List[SelectFromModel], SelectFromModel] = None,
#     oversample: bool = False,
#     random_state: int = None
# ):

#StratifiedKFold
def cross_val(
    X: pd.DataFrame,
    y: np.ndarray,
    groups: np.ndarray,
    path: str,
    name: str,
    estimator: BaseEstimator,
    categories: List[str] = None,
    normalize: bool = False,
    split: str = None,
    split_params: Dict[str, any] = None,
    select: Union[List[SelectFromModel], SelectFromModel] = None,
    oversample: bool = False,
    random_state: int = None
):
    if not os.path.exists(path):
        raise ValueError('"path" does not exist.')
    
    if not split:
        raise ValueError('"split" should be specified.')
    
    if not ray.is_initialized():
        raise EnvironmentError('"ray" should be initialized.')
    
    jobs = []
    func = ray.remote(_train).remote

    categories = list() if categories is None else categories
    C_cat = np.asarray(sorted(categories))
    C_num = np.asarray(sorted(X.columns[~X.columns.isin(C_cat)]))

    split_params = split_params or dict()
    splitter = _split(alg=split, X=X, y=y, groups=groups, random_state=random_state, **split_params)

    for idx_fold, (I_train, I_test) in enumerate(splitter):
        if split == 'logo':
            FOLD_NAME = str(np.unique(groups[I_test]).item(0))
        else:
            FOLD_NAME = f"fold{idx_fold + 1}"

        X_train, y_train = X.iloc[I_train, :], y[I_train]
        X_test, y_test = X.iloc[I_test, :], y[I_test]

        job = func(
            dir_result=path,
            name=f'{name}#{FOLD_NAME}',
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            y_test=y_test,
            C_cat=C_cat,
            C_num=C_num,
            categories=categories,
            estimator=clone(estimator),
            normalize=normalize,
            select=select,
            oversample=oversample,
            random_state=random_state
        )
        jobs.append(job)
    ray.get(jobs)

### Minor Modification on XGBClassifer
This modification allows XGBClassifiers to automatically generate evaluation sets during pipeline (without passing any argument in "fit" function)

In [56]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator
from sklearn.model_selection import StratifiedShuffleSplit
from typing import Union


class EvXGBClassifier(BaseEstimator):
    def __init__(
        self,
        eval_size=None,
        eval_metric='logloss',
        early_stopping_rounds=10,
        random_state=None,
        **kwargs
        ):
        self.random_state = random_state
        self.eval_size = eval_size
        self.eval_metric = eval_metric
        self.early_stopping_rounds = early_stopping_rounds
        self.model = XGBClassifier(
            random_state=self.random_state,
            eval_metric=self.eval_metric,
            early_stopping_rounds=self.early_stopping_rounds,
            **kwargs
        )

    @property
    def classes_(self):
        return self.model.classes_

    @property
    def feature_importances_(self):
        return self.model.feature_importances_
    
    @property
    def feature_names_in_(self):
        return self.model.feature_names_in_

    def fit(self, X: Union[pd.DataFrame, np.ndarray], y: np.ndarray):
        if self.eval_size:
            splitter = StratifiedShuffleSplit(random_state=self.random_state, test_size=self.eval_size)
            I_train, I_eval = next(splitter.split(X, y))
            if isinstance(X, pd.DataFrame):
                X_train, y_train = X.iloc[I_train, :], y[I_train]
                X_eval, y_eval = X.iloc[I_eval, :], y[I_eval]
            else:
                X_train, y_train = X[I_train, :], y[I_train]
                X_eval, y_eval = X[I_eval, :], y[I_eval]
                
            self.model = self.model.fit(
                X=X_train, y=y_train, 
                eval_set=[(X_eval, y_eval)],
                verbose=False
            )
        else:
            self.model = self.model.fit(X=X, y=y, verbose=False)
        return self

    def predict(self, X: pd.DataFrame):
        return self.model.predict(X)

    def predict_proba(self, X: pd.DataFrame):
        return self.model.predict_proba(X)

## Execution

Unfortunately, our feature data has a big-$p$, little-$N$ problem: # sample = 2,619 while # features = 3,356.
Therefore, we need to choose important features only. 

In [57]:
# pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [58]:
# import sklearn
# sklearn.__version__

In [36]:
# pip uninstall lightgbm pandas dask

In [37]:
# pip install pandas==2.0.3 lightgbm dask

In [ ]:
import os
from itertools import product
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import VotingClassifier

# LightGBM import
from lightgbm import LGBMClassifier

lgbm_native = LGBMClassifier(
    random_state=42,
    learning_rate=0.03,
    max_depth=5,               
    num_leaves=31,              
    min_child_samples=10,     
    min_split_gain=0.0,         
    reg_alpha=2.0,
    reg_lambda=2.0,
    subsample=0.8,
    colsample_bytree=0.8,
    n_estimators=100,
    importance_type='gain',
    verbosity=-1               
)

RANDOM_STATE = 42

ESTIMATOR_LOGREG = LogisticRegression(
    penalty='l1', solver='liblinear', random_state=RANDOM_STATE
)
ESTIMATOR_RF = RandomForestClassifier(random_state=RANDOM_STATE, max_depth=5)

ESTIMATOR_XGB = EvXGBClassifier(
    random_state=RANDOM_STATE,
    eval_metric='logloss',
    eval_size=0.2,
    early_stopping_rounds=50,
    objective='binary:logistic',
    learning_rate=0.03,
    reg_alpha=2.0,
    reg_lambda=2.0,
    gamma=2.0,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8
)

ESTIMATOR_ENSEMBLE = VotingClassifier(
    estimators=[
        ('logreg', clone(ESTIMATOR_LOGREG)),
        ('rf', clone(ESTIMATOR_RF)),
        ('lgbm', clone(lgbm_native)) 
    ],
    voting='soft'
)

SELECT_SVC = SelectFromModel(
    estimator=LinearSVC(
        penalty='l1',
        loss='squared_hinge',
        dual=False,
        tol=1e-3,
        C=1e-2,
        max_iter=5000,
        random_state=RANDOM_STATE
    ),
    threshold=1e-5
)

CLS = ['attention_train_val']  # train, test 용으로 수정
SETTINGS = [
    dict(
        estimator=clone(ESTIMATOR_LOGREG),
        oversample=False,
        select=None,
        name='logreg'
    ),
    dict(
        estimator=clone(ESTIMATOR_RF),
        oversample=False,
        select=[clone(SELECT_SVC)],
        name='rf_ns'
    ),
    dict(
        estimator=clone(ESTIMATOR_ENSEMBLE),
        oversample=False,
        select=[clone(SELECT_SVC)],
        name='voting_ensemble'
    ),
    dict(
        estimator=clone(ESTIMATOR_XGB),
        oversample=False,
        select=[clone(SELECT_SVC)],
        name='xgb_ns'
    ),
    # 새로 추가: LightGBM 단독 실험용
    dict(
        estimator=clone(lgbm_native),
        oversample=False,
        select=[clone(SELECT_SVC)],
        name='lgbm_ns'
    ),
]

with on_ray(num_cpus=12):
    for l, s in product(
        CLS, SETTINGS
    ):
        p = os.path.join(PATH_INTERMEDIATE, f'{l}.pkl')
        par_dir = os.path.join(PATH_INTERMEDIATE, 'eval', l)
        os.makedirs(par_dir, exist_ok=True)

        X, y, groups, t, datetimes = load(p)
        print(groups)
        cats = X.columns[X.dtypes == bool]
        cross_val(
            X=X, y=y, groups=groups,
            path=par_dir,
            categories=cats,
            normalize=True,
            split='stratifiedgroupkfold',
            split_params={'n_splits': 5},  # logo시 삭제
            random_state=RANDOM_STATE,
            **s
        )

2025-06-05 23:00:50,747	INFO worker.py:1852 -- Started a local Ray instance.


['P50' 'P49' 'P45' ... 'P08' 'P15' 'P69']
(_train pid=16380) [25-06-05 23:01:02] In progress: [logreg#fold1] Normalizing numeric features.
(_train pid=23912) [25-06-05 23:01:02] Success: [logreg#fold4] Normalizing numeric features (0.15s).
(_train pid=23912) [25-06-05 23:01:02] In progress: [logreg#fold4] Training.
(_train pid=18052) [25-06-05 23:01:09] Success: [logreg#fold2] Training (7.44s).
(_train pid=23912) [25-06-05 23:01:02] In progress: [logreg#fold4] Normalizing numeric features. [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_train pid=18052) [25-06-05 23:01:02] Success: [logreg#fold2] Normalizing numeric features (0.20s). [repeated 4x across cluster]
(_train pid=18052) [25-06-05 23:01:02] In progress: [logreg#fold2] Training. [repeated 4x across cluster]
['P50' 'P49' 'P45' ... 'P08

In [60]:
# StratifiedGroupKFold 잘 되었는지 확인
from sklearn.model_selection import StratifiedGroupKFold
import numpy as np

sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

for fold_idx, (train_idx, test_idx) in enumerate(sgkf.split(X, y, groups=groups)):
    train_groups = np.unique(groups[train_idx])
    test_groups = np.unique(groups[test_idx])
    intersection = np.intersect1d(train_groups, test_groups)
    
    print(f"Fold {fold_idx + 1}")
    print(f"  Train groups: {train_groups}")
    print(f"  Test groups:  {test_groups}")
    
    if len(intersection) == 0:
        print("OK: No overlap between train/test groups\n")
    else:
        print(f"Overlap found: {intersection}\n")

Fold 1
  Train groups: ['P03' 'P05' 'P06' 'P08' 'P09' 'P10' 'P12' 'P15' 'P19' 'P21' 'P23' 'P26'
 'P28' 'P30' 'P31' 'P33' 'P35' 'P40' 'P42' 'P45' 'P47' 'P48' 'P50' 'P51'
 'P52' 'P53' 'P57' 'P60' 'P61' 'P66' 'P67' 'P69' 'P70' 'P72' 'P75' 'P77'
 'P78' 'P79' 'P80']
  Test groups:  ['P01' 'P02' 'P13' 'P32' 'P39' 'P49' 'P55' 'P76']
OK: No overlap between train/test groups

Fold 2
  Train groups: ['P01' 'P02' 'P03' 'P05' 'P06' 'P10' 'P12' 'P13' 'P19' 'P21' 'P23' 'P26'
 'P30' 'P31' 'P32' 'P33' 'P39' 'P40' 'P47' 'P48' 'P49' 'P50' 'P51' 'P52'
 'P53' 'P55' 'P60' 'P66' 'P67' 'P69' 'P70' 'P72' 'P75' 'P76' 'P77' 'P78'
 'P80']
  Test groups:  ['P08' 'P09' 'P15' 'P28' 'P35' 'P42' 'P45' 'P57' 'P61' 'P79']
OK: No overlap between train/test groups

Fold 3
  Train groups: ['P01' 'P02' 'P06' 'P08' 'P09' 'P10' 'P13' 'P15' 'P19' 'P21' 'P26' 'P28'
 'P30' 'P32' 'P33' 'P35' 'P39' 'P40' 'P42' 'P45' 'P47' 'P48' 'P49' 'P50'
 'P51' 'P52' 'P55' 'P57' 'P61' 'P66' 'P67' 'P72' 'P76' 'P77' 'P78' 'P79'
 'P80']
  Test gro

# Evaluation

## Implementation

In [61]:
import numpy as np
from typing import Dict
from itertools import product
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, balanced_accuracy_score, \
    confusion_matrix, precision_recall_fscore_support, \
    roc_auc_score, matthews_corrcoef, average_precision_score, \
    log_loss, brier_score_loss
import scipy.stats.mstats as ms


def evaluate(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    y_proba: np.ndarray,
    classes: np.ndarray
) -> Dict[str, any]:
    R = {}
    n_classes = len(classes)
    is_multiclass = n_classes > 2
    is_same_y = len(np.unique(y_true)) == 1
    R['inst'] = len(y_true)
    
    for c in classes:
        R[f'inst_{c}'] = np.sum(y_true == c)
        
    if not is_multiclass:
        _, cnt = np.unique(y_true, return_counts=True)
        
        if len(cnt) > 1:
            R['class_ratio'] = cnt[0] / cnt[1]
        else:
            R['class_ratio'] = np.nan

    C = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=classes)
    for (i1, c1), (i2, c2) in product(enumerate(classes), enumerate(classes)):
        R[f'true_{c1}_pred_{c2}'] = C[i1, i2]

    # Threshold Measure
    R['acc'] = accuracy_score(y_true=y_true, y_pred=y_pred)
    R['bac'] = balanced_accuracy_score(y_true=y_true, y_pred=y_pred)
    R['gmean'] = ms.gmean(np.diag(C) / np.sum(C, axis=1))
    R['mcc'] = matthews_corrcoef(y_true=y_true, y_pred=y_pred)
    
    if is_multiclass:
        for avg in ('macro', 'micro'):
            pre, rec, f1, _ = precision_recall_fscore_support(
                y_true=y_true,
                y_pred=y_pred,
                labels=classes,
                average=avg, 
                zero_division=0
            )
            R[f'pre_{avg}'] = pre
            R[f'rec_{avg}'] = rec
            R[f'f1_{avg}'] = f1
    else:
        pre, rec, f1, _ = precision_recall_fscore_support(
            y_true=y_true, y_pred=y_pred, pos_label=c, average='macro', zero_division=0
        )
        R[f'pre_macro'] = pre
        R[f'rec_macro'] = rec
        R[f'f1_macro'] = f1
        
        for c in classes:
            pre, rec, f1, _ = precision_recall_fscore_support(
                y_true=y_true, y_pred=y_pred, pos_label=c, average='binary', zero_division=0
            )
            R[f'pre_{c}'] = pre
            R[f'rec_{c}'] = rec
            R[f'f1_{c}'] = f1

    # Ranking Measure
    if is_multiclass:
        for avg, mc in product(('macro', 'micro'), ('ovr', 'ovo')):
            R[f'roauc_{avg}_{mc}'] = roc_auc_score(
                y_true=y_true, y_score=y_proba,
                average=avg, multi_class=mc, labels=classes
            ) if not is_same_y else np.nan
    else:
        R[f'roauc'] = roc_auc_score(
            y_true=y_true, y_score=y_proba[:, 1], average=None
        ) if not is_same_y else np.nan
        for i, c in enumerate(classes):
            R[f'prauc_{c}'] = average_precision_score(
                y_true=y_true, y_score=y_proba[:, i], pos_label=c, average=None
            ) 
            R[f'prauc_ref_{c}'] = np.sum(y_true == c) / len(y_true)

    # Probability Measure
    R['log_loss'] = log_loss(y_true=y_true, y_pred=y_proba, labels=classes, normalize=True)

    if not is_multiclass:
        R[f'brier_loss'] = brier_score_loss(
            y_true=y_true, y_prob=y_proba[:, 1], pos_label=classes[1]
        )

    return R

## Execution

In [62]:
import os
import pandas as pd


RESULTS_EVAL = []
DIR_EVAL = os.path.join(PATH_INTERMEDIATE, 'eval')

for l in ['attention_train_val']:
    dir_l = os.path.join(DIR_EVAL, l)
    if not os.path.exists(dir_l):
        continue
    
    for f in os.listdir(dir_l):
        model, pid = f[:f.index('.pkl')].split('#')
        res = load(os.path.join(dir_l, f))
        X, y = res.X_test, res.y_test
        y_pred = res.estimator.predict(X)
        y_proba = res.estimator.predict_proba(X)
        ev_test = evaluate(
            y_true=y,
            y_pred=y_pred,
            y_proba=y_proba,
            classes=[0, 1]
        )

        X, y = res.X_train, res.y_train
        y_pred = res.estimator.predict(X)
        y_proba = res.estimator.predict_proba(X)
        ev_train = evaluate(
            y_true=y,
            y_pred=y_pred,
            y_proba=y_proba,
            classes=[0, 1]
        )

        RESULTS_EVAL.append({
            'label': l,
            'alg': model,
            'split': pid,
            'n_feature': len(X.columns),
            **{
                f'test_{k}': v for k, v in ev_test.items()
            },
            **{
                f'train_{k}': v for k, v in ev_train.items()
                            }
        })
    
RESULTS_EVAL = pd.DataFrame(RESULTS_EVAL)
RESULTS_EVAL

c:\Users\leejo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:3284: FutureWarning: y_prob was deprecated in version 1.5 and will be removed in 1.7.Please use ``y_proba`` instead.
  warnings.warn(
c:\Users\leejo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:3284: FutureWarning: y_prob was deprecated in version 1.5 and will be removed in 1.7.Please use ``y_proba`` instead.
  warnings.warn(
c:\Users\leejo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:3284: FutureWarning: y_prob was deprecated in version 1.5 and will be removed in 1.7.Please use ``y_proba`` instead.
  warnings.warn(
c:\Users\leejo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:3284: FutureWarning: y_prob was deprecated in version 1.5 and will be removed in 1.7.Please use ``y_proba`` instead.
  warnings.warn(
c:\Users\leejo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:3284: FutureWarning: y_prob was deprecated in version 1.5 and will be remo

,label,alg,split,n_feature,test_inst,test_inst_0,test_inst_1,test_class_ratio,test_true_0_pred_0,test_true_0_pred_1,...,train_pre_1,train_rec_1,train_f1_1,train_roauc,train_prauc_0,train_prauc_ref_0,train_prauc_1,train_prauc_ref_1,train_log_loss,train_brier_loss
0,attention_train_val,logreg,fold1,84,8661,4565,4096,1.114502,3343,1222,...,0.744885,0.749151,0.747012,0.843067,0.847742,0.495421,0.844084,0.504579,0.490646,0.163573
1,attention_train_val,logreg,fold2,84,10753,5495,5258,1.045074,4319,1176,...,0.727539,0.729222,0.728380,0.822500,0.832018,0.498168,0.818274,0.501832,0.516201,0.173482
2,attention_train_val,logreg,fold3,84,10682,5819,4863,1.196586,4271,1548,...,0.750579,0.751600,0.751089,0.845009,0.848693,0.488909,0.848522,0.511091,0.490071,0.163238
3,attention_train_val,logreg,fold4,84,8572,4825,3747,1.287697,3615,1210,...,0.743664,0.747745,0.745699,0.836728,0.839812,0.487991,0.838956,0.512009,0.499651,0.166805
4,attention_train_val,logreg,fold5,84,10831,4093,6738,0.607450,2604,1489,...,0.733871,0.708584,0.721006,0.842954,0.868166,0.535430,0.821095,0.464570,0.491854,0.164005
5,attention_train_val,rf_ns,fold1,66,8661,4565,4096,1.114502,2826,1739,...,0.732887,0.930069,0.819788,0.896452,0.905178,0.495421,0.898846,0.504579,0.505015,0.162921
6,attention_train_val,rf_ns,fold2,62,10753,5495,5258,1.045074,3830,1665,...,0.713060,0.923524,0.804759,0.877666,0.890774,0.498168,0.877044,0.501832,0.529937,0.173736
7,attention_train_val,rf_ns,fold3,55,10682,5819,4863,1.196586,2899,2920,...,0.715432,0.962095,0.820629,0.904745,0.908681,0.488909,0.910938,0.511091,0.457849,0.147218
8,attention_train_val,rf_ns,fold4,62,8572,4825,3747,1.287697,2845,1980,...,0.725209,0.910188,0.807237,0.891363,0.896484,0.487991,0.898472,0.512009,0.514070,0.166813
9,attention_train_val,rf_ns,fold5,61,10831,4093,6738,0.607450,2003,2090,...,0.722133,0.873803,0.790761,0.893221,0.914728,0.535430,0.879047,0.464570,0.503109,0.163301


In [63]:
import pandas as pd


SUMMARY_EVAL = []

for row in RESULTS_EVAL.groupby(
    ['label', 'alg']
).agg(summary).reset_index().itertuples():
    for k, v in row._asdict().items():
        if type(v) is dict:
            r = dict(
                label=row.label,
                alg=row.alg,
                metric=k,
                **v
            )
            SUMMARY_EVAL.append(r)

SUMMARY_EVAL = pd.DataFrame(SUMMARY_EVAL)    
SUMMARY_EVAL

,label,alg,metric,n,cardinality,value_count,sum,mean,SD,med,range,conf.,nan_count
0,attention_train_val,logreg,split,5,5.0,"fold1:1, fold2:1, fold3:1, fold4:1, fold5:1",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,attention_train_val,logreg,n_feature,5,NaN,NaN,420.000000,84.000000,0.000000,84.000000,"(84, 84)","(nan, nan)",0.0
2,attention_train_val,logreg,test_inst,5,NaN,NaN,49499.000000,9899.800000,1173.094071,10682.000000,"(8572, 10831)","(8443.211325223596, 11356.388674776403)",0.0
3,attention_train_val,logreg,test_inst_0,5,NaN,NaN,24797.000000,4959.400000,698.219736,4825.000000,"(4093, 5819)","(4092.445690742198, 5826.354309257801)",0.0
4,attention_train_val,logreg,test_inst_1,5,NaN,NaN,24702.000000,4940.400000,1169.983034,4863.000000,"(3747, 6738)","(3487.674188092248, 6393.125811907751)",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,attention_train_val,xgb_ns,train_prauc_ref_0,5,NaN,NaN,2.505919,0.501184,0.019620,0.495421,"(0.4879908129107924, 0.5354298127650771)","(0.4768220083960241, 0.52554543494659)",0.0
228,attention_train_val,xgb_ns,train_prauc_1,5,NaN,NaN,4.993768,0.998754,0.000144,0.998774,"(0.9985745741181193, 0.9989060681626198)","(0.9985745534754127, 0.9989326101825202)",0.0
229,attention_train_val,xgb_ns,train_prauc_ref_1,5,NaN,NaN,2.494081,0.498816,0.019620,0.504579,"(0.46457018723492294, 0.5120091870892076)","(0.47445456505341005, 0.5231779916039759)",0.0
230,attention_train_val,xgb_ns,train_log_loss,5,NaN,NaN,0.252160,0.050432,0.002487,0.049696,"(0.048078138330567274, 0.05330088633377527)","(0.047344179980161605, 0.05351983509387892)",0.0


Below shows metrics of our interest only.

In [64]:
SUB_SUMMARY_EVAL = SUMMARY_EVAL.loc[
    lambda x: x['metric'].isin(
        ['n_feature', 'train_class_ratio', 'train_inst_0', 'train_inst_1', 'test_inst_0', 'test_inst_1', 'test_acc', 'test_f1_0' ,'test_f1_1', 'test_f1_macro', 'train_f1_0' ,'train_f1_1', 'train_f1_macro',]
    )
].round(3).assign(
    mean_sd=lambda x: x['mean'].astype(str).str.cat(' (' + x['SD'].astype(str) + ')', sep='')
).pivot(
    index=['label', 'alg'], columns=['metric'], values=['mean_sd']
)
SUB_SUMMARY_EVAL.to_csv('./fig/SUB_SUMMARY_EVAL.csv')
SUB_SUMMARY_EVAL

mean_sd                 \
metric                                  n_feature       test_acc   
label               alg                                            
attention_train_val logreg             84.0 (0.0)  0.729 (0.033)   
                    rf_ns            61.2 (3.962)   0.73 (0.032)   
                    voting_ensemble  61.2 (3.962)  0.729 (0.029)   
                    xgb_ns           61.2 (3.962)  0.737 (0.023)   

                                                                   \
metric                                   test_f1_0      test_f1_1   
label               alg                                             
attention_train_val logreg           0.726 (0.062)  0.722 (0.048)   
                    rf_ns            0.678 (0.071)  0.761 (0.035)   
                    voting_ensemble  0.714 (0.059)  0.734 (0.043)   
                    xgb_ns            0.729 (0.05)  0.736 (0.044)   

                                                                     \
metric                               test_f1_macro      test_inst_0   
label               alg                                               
attention_train_val logreg           0.724 (0.036)  4959.4 (698.22)   
                    rf_ns            0.719 (0.037)  4959.4 (698.22)   
                    voting_ensemble  0.724 (0.032)  4959.4 (698.22)   
                    xgb_ns           0.733 (0.025)  4959.4 (698.22)   

                                                                          \
metric                                     test_inst_1 train_class_ratio   
label               alg                                                    
attention_train_val logreg           4940.4 (1169.983)     1.007 (0.083)   
                    rf_ns            4940.4 (1169.983)     1.007 (0.083)   
                    voting_ensemble  4940.4 (1169.983)     1.007 (0.083)   
                    xgb_ns           4940.4 (1169.983)     1.007 (0.083)   

                                                                   \
metric                                  train_f1_0     train_f1_1   
label               alg                                             
attention_train_val logreg           0.741 (0.015)  0.739 (0.013)   
                    rf_ns             0.748 (0.02)  0.809 (0.012)   
                    voting_ensemble  0.744 (0.016)  0.763 (0.015)   
                    xgb_ns             0.996 (0.0)    0.996 (0.0)   

                                                                      \
metric                              train_f1_macro      train_inst_0   
label               alg                                                
attention_train_val logreg            0.74 (0.007)  19837.6 (698.22)   
                    rf_ns            0.778 (0.008)  19837.6 (698.22)   
                    voting_ensemble  0.754 (0.007)  19837.6 (698.22)   
                    xgb_ns             0.996 (0.0)  19837.6 (698.22)   

                                                         
metric                                     train_inst_1  
label               alg                                  
attention_train_val logreg           19761.6 (1169.983)  
                    rf_ns            19761.6 (1169.983)  
                    voting_ensemble  19761.6 (1169.983)  
                    xgb_ns           19761.6 (1169.983)

In [65]:
import os
import joblib
import pandas as pd
from sklearn.metrics import f1_score

MODEL_NAMES = ['lgbm_ns', 'voting_ensemble','logreg', 'rf_ns', 'xgb_ns']
RESULTS = {}

for model_name in MODEL_NAMES:
    scores = []
    for fold in range(1, 6):
        path = f'intermediate/eval/attention/{model_name}#fold{fold}.pkl'
        if os.path.exists(path):
            result = joblib.load(path)
            y_true, y_pred = result.y_test, result.estimator.predict(result.X_test)
            score = f1_score(y_true, y_pred, average='macro')
            scores.append(score)
    RESULTS[model_name] = scores

df_scores = pd.DataFrame(RESULTS)
df_scores.to_csv("foldwise_f1_scores.csv", index=False)
print(df_scores)

Empty DataFrame
Columns: [lgbm_ns, voting_ensemble, logreg, rf_ns, xgb_ns]
Index: []


In [66]:
from scipy.stats import ttest_rel, f_oneway
from itertools import combinations
import pandas as pd

print("Paired t-test:")

# 귀무가설: 두 모델의 성능 차이는 0이다. (성능이 동일하다.)
for model_a, model_b in combinations(df_scores.columns, 2):
    stat, p = ttest_rel(df_scores[model_a], df_scores[model_b])
    print(f"{model_a} vs {model_b} → t = {stat:.4f}, p = {p:.4f} {'*' if p < 0.05 else ''}")

print("\nANOVA:")
f_stat, p_value = f_oneway(*[df_scores[col] for col in df_scores.columns])
print(f"F = {f_stat:.4f}, p = {p_value:.4f} {'*' if p_value < 0.05 else ''}")

Paired t-test:
lgbm_ns vs voting_ensemble → t = nan, p = nan 
lgbm_ns vs logreg → t = nan, p = nan 
lgbm_ns vs rf_ns → t = nan, p = nan 
lgbm_ns vs xgb_ns → t = nan, p = nan 
voting_ensemble vs logreg → t = nan, p = nan 
voting_ensemble vs rf_ns → t = nan, p = nan 
voting_ensemble vs xgb_ns → t = nan, p = nan 
logreg vs rf_ns → t = nan, p = nan 
logreg vs xgb_ns → t = nan, p = nan 
rf_ns vs xgb_ns → t = nan, p = nan 

ANOVA:
F = nan, p = nan 


C:\Users\leejo\AppData\Local\Temp\ipykernel_7760\1869363743.py:9: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p = ttest_rel(df_scores[model_a], df_scores[model_b])
C:\Users\leejo\AppData\Local\Temp\ipykernel_7760\1869363743.py:13: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  f_stat, p_value = f_oneway(*[df_scores[col] for col in df_scores.columns])


In [67]:
import os
import joblib
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from typing import Dict

MODEL_NAMES = ['logreg', 'rf_ns', 'xgb_ns']
N_FOLDS = 5

metrics: Dict[str, Dict[str, list]] = {name: {'f1': [], 'acc': [], 'auc': []} for name in MODEL_NAMES}

for model_name in MODEL_NAMES:
    for fold in range(1, N_FOLDS + 1):
        path = f"intermediate/eval/attention/{model_name}#fold{fold}.pkl"
        if os.path.exists(path):
            result = joblib.load(path)
            y_true = result.y_test
            y_pred = result.estimator.predict(result.X_test)

            # predict_proba 사용 가능하면 AUC 계산
            if hasattr(result.estimator, "predict_proba"):
                y_prob = result.estimator.predict_proba(result.X_test)[:, 1]
                auc = roc_auc_score(y_true, y_prob)
            else:
                auc = float('nan')

            metrics[model_name]['f1'].append(f1_score(y_true, y_pred, average='macro'))
            metrics[model_name]['acc'].append(accuracy_score(y_true, y_pred))
            metrics[model_name]['auc'].append(auc)

df_f1 = pd.DataFrame({m: metrics[m]['f1'] for m in MODEL_NAMES})
df_acc = pd.DataFrame({m: metrics[m]['acc'] for m in MODEL_NAMES})
df_auc = pd.DataFrame({m: metrics[m]['auc'] for m in MODEL_NAMES})

df_f1.to_csv("foldwise_f1.csv", index=False)
df_acc.to_csv("foldwise_accuracy.csv", index=False)
df_auc.to_csv("foldwise_auc.csv", index=False)

In [68]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

os.makedirs('./fig/confusion_matrices', exist_ok=True)

cm_metrics = ['true_0_pred_0', 'true_0_pred_1', 'true_1_pred_0', 'true_1_pred_1']
f1_metrics = ['f1_macro', 'f1_micro']
cm_metrics = [f'test_{m}' for m in cm_metrics]
f1_metrics = [f'test_{m}' for m in f1_metrics]

mean_cm_df = SUMMARY_EVAL[SUMMARY_EVAL['metric'].isin(cm_metrics)].pivot(index=['label', 'alg'], columns='metric', values='mean')
std_cm_df = SUMMARY_EVAL[SUMMARY_EVAL['metric'].isin(cm_metrics)].pivot(index=['label', 'alg'], columns='metric', values='SD')
f1_df = SUMMARY_EVAL[SUMMARY_EVAL['metric'].isin(f1_metrics)].pivot(index=['label', 'alg'], columns='metric', values='mean')

for idx in mean_cm_df.index:
    try:
        mean_cm = np.array([
            [mean_cm_df.loc[idx, 'test_true_0_pred_0'], mean_cm_df.loc[idx, 'test_true_0_pred_1']],
            [mean_cm_df.loc[idx, 'test_true_1_pred_0'], mean_cm_df.loc[idx, 'test_true_1_pred_1']]
        ])
        std_cm = np.array([
            [std_cm_df.loc[idx, 'test_true_0_pred_0'], std_cm_df.loc[idx, 'test_true_0_pred_1']],
            [std_cm_df.loc[idx, 'test_true_1_pred_0'], std_cm_df.loc[idx, 'test_true_1_pred_1']]
        ])
        annotations = np.empty_like(mean_cm, dtype=object)
        for i in range(2):
            for j in range(2):
                annotations[i, j] = f"{mean_cm[i, j]:.1f}±{std_cm[i, j]:.1f}"

        macro_f1 = f1_df.loc[idx, 'test_f1_macro']
        micro_f1 = f1_df.loc[idx, 'test_f1_micro'] if 'test_f1_micro' in f1_df.columns else None

        plt.figure(figsize=(6, 5))
        sns.heatmap(mean_cm, annot=annotations, fmt='', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
        plt.title(f'Confusion Matrix: {idx[0]} | {idx[1]} (mean ± std)')
        plt.xlabel('Predicted label')
        plt.ylabel('True label')

        f1_text = f"Macro F1: {macro_f1:.3f}"
        if micro_f1 is not None:
            f1_text += f" | Micro F1: {micro_f1:.3f}"
        plt.figtext(0.5, -0.05, f1_text, ha='center', fontsize=10)

        # 저장
        filename = f"confmat_{idx[0]}_{idx[1]}.png".replace(" ", "_")
        filepath = os.path.join('./fig/confusion_matrices', filename)
        plt.tight_layout()
        plt.savefig(filepath, dpi=300, bbox_inches='tight')
        plt.close()

    except Exception as e:
        print(f"Skipping {idx} due to error: {e}")

# Feature Importances

## Implementation

In [69]:
from typing import Union, Optional


def feature_importance(
    estimator
):
    if not hasattr(estimator, 'feature_names_in_') or not hasattr(estimator, 'feature_importances_'):
        return None
    
    names = estimator.feature_names_in_
    importances = estimator.feature_importances_
    
    return names, importances

## Execution

In [140]:
import os
import pandas as pd
from collections import defaultdict


IMPORTANCE_EVAL = defaultdict(list)
DIR_EVAL = os.path.join(PATH_INTERMEDIATE, 'eval')

for l in ['attention_train_val']:
    dir_l = os.path.join(DIR_EVAL, l)
    if not os.path.exists(dir_l):
        continue
    
    for f in os.listdir(dir_l):
        res = load(os.path.join(dir_l, f))

        f_norm = f[:f.index('.pkl')]
        alg = f_norm[:f.rindex('#')]
        
        feat_imp = feature_importance(res.estimator)
        if not feat_imp:
            continue
            
        names, importance = feat_imp
        new_names = []
        for n in names:
            for c in res.categories:
                n = n.replace(f'{c}_', f'{c}=')
            new_names.append(n)
        
        d = pd.DataFrame(
            importance.reshape(1, -1),
            columns=new_names
        )
        IMPORTANCE_EVAL[(l, alg)].append(d)
        

IMPORTANCE_SUMMARY = []

for (l, alg), v in IMPORTANCE_EVAL.items():
    new_v = pd.concat(
        v, axis=0
    ).fillna(0.0).mean().reset_index().set_axis(
        ['feature', 'importance'], axis=1
    ).assign(
        label=l,
        alg=alg
    )
    IMPORTANCE_SUMMARY.append(new_v)
    
IMPORTANCE_SUMMARY = pd.concat(IMPORTANCE_SUMMARY, axis=0, ignore_index=True)


### Plot

In [141]:
%%R -i IMPORTANCE_SUMMARY -w 26 -h 16 -u cm

library(ggplot2)
library(dplyr)
library(stringr)
library(patchwork)

data <- IMPORTANCE_SUMMARY %>% filter(label == 'attention')

p_label <- ggplot() + geom_text(
    aes(x = .5, y = .5),
    label = 'Attention',
    family = 'ssp',
    fontface = 'bold',
    size = 4
) + theme_void()

p_rf <- ggplot(
    data %>% filter(alg == 'rf_os') %>% top_n(n = 10, wt = importance),
    aes(x = reorder(feature, -importance), y = importance)
) + geom_col() +
    THEME_DEFAULT + theme(
        axis.text.x = element_text(angle = 90, size = 10, hjust = 1, vjust = .5),
        axis.title.x = element_blank(),
        axis.title.y = element_blank()
    ) + labs(subtitle = 'Random Forest')

p_xgb <- ggplot(
    data %>% filter(alg == 'xgb_os') %>% top_n(n = 10, wt = importance),
    aes(x = reorder(feature, -importance), y = importance)
) + geom_col() +
    THEME_DEFAULT + theme(
        axis.text.x = element_text(angle = 90, size = 10, hjust = 1, vjust = .5),
        axis.title.x = element_blank(),
        axis.title.y = element_blank()
    ) + labs(subtitle = 'XGBoost')

p <- p_label / (p_rf | p_xgb) + plot_layout(heights = c(1.1, 10))

ggsave('./fig/imp_attention.pdf', plot = p, width = 26, height = 16, unit = 'cm', device = cairo_pdf)
print(p)

UsageError: Cell magic `%%R` not found.
